In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE social_media_bronze
AS
SELECT * 
FROM cloud_files('${source}/random_events', 'json', map('cloudFIles.inferColumnTypes', 'true'))

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE social_media_silver
AS
SELECT
    post_id,
    platform,
    post_timestamp,
    SUM(CASE WHEN event_type = 'like' THEN 1 ELSE 0 END) AS likes,
    SUM(CASE WHEN event_type = 'comment' THEN 1 ELSE 0 END) AS comments,
    SUM(CASE WHEN event_type = 'share' THEN 1 ELSE 0 END) AS shares,
    0.25 * (
        SUM(CASE WHEN event_type = 'like' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN event_type = 'comment' THEN 1 ELSE 0 END) +
        SUM(CASE WHEN event_type = 'share' THEN 1 ELSE 0 END)
    ) AS reach
FROM STREAM(LIVE.social_media_bronze)
GROUP BY post_id, platform;

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE social_media_gold
AS
SELECT 
    post_id, 
    platform,
    post_timestamp, 
    to_date(date_format(post_timestamp, 'yyyy-MM-dd')) AS post_date, 
    date_format(post_timestamp, 'HH:mm:ss') AS post_time,
    likes, 
    comments, 
    shares, 
    reach,
    (COALESCE(likes, 0) + COALESCE(shares, 0) + COALESCE(comments, 0)) AS total_engagements,
    CASE 
        WHEN COALESCE(reach, 0) > 0 THEN 
            ((COALESCE(likes, 0) + COALESCE(shares, 0) + COALESCE(comments, 0)) / COALESCE(reach, 1)) * 100
        ELSE 0
    END AS engagement_rate
FROM STREAM(LIVE.social_media_silver);
